# ADP Projekt

## Łukasz Gałuszewski, Mateusz Dominiak, Szymon Giergiel

---

### Cele projektu:
1. Analiza rozmieszczenie populacji:
    - Zbadanie rozmieszczenia ludności w różnych województwach w Polsce.
2. Zbadanie rozmieszczenia liczby szkół:
    - Oceń liczbę szkół w każdym województwie i ich rozmieszczenie.
3. Obliczenie liczby szkół przypadających na mieszkańca:
    - Określenie liczby szkół przypadającą na mieszkańca w każdym województwie.
4. Porównanie szkół publicznych i prywatnych:
    - Przeanalizowanie odsetek szkół publicznych i prywatnych w każym województwie.
5. Mapa rozmieszczenia szkół:
    - Stworzenie mapy przedstawiającej rozmieszczenie geograficzne szkół i zidentyfikowanie obszarów, w których w promieniu 15 km nie ma szkół średnich.
6. Mapa cieplna gęstości zaludnienia studentów:
    - Wygenerowanie mapy cieplnej, aby pokazać odróżnić obszary o dużej i niskiej gęstości zaludnienia uczniów.
7. Ocena odległości do najlepiej ocenianych liceów w Polsce:
    - Określenie odległości do najlepiej ocenianych szkół średnich i zidentyfikowanie obszarów, które mają słaby dostęp do lepszych szkół.

### Hipotetyczny problem:
Analiza infrastruktury edukacyjnej odkrywa znaczne dysproporcje w rozmieszczeniu i dostępie do szkół wyższych i wysokiej jakości edukacji. W niektórych województwach liczba szkół przypadających na mieszkańca jest relatywnie wysoka, podczas gdy w innych jest znacząco niższa, co może świadczyć o problemach z równym dostępem do edukacji. Ponadto rozmieszczenie geograficzne szkół pokazuje, że niektóre szkoły podstawowe nie mają szkół średnich w promieniu 15 kilometrów, co może być znaczną niedogodnością dla uczniów i ich rodzin. Ten brak pobliskich szkół średnich może mieć wpływ na ciągłość edukacji uczniów i dostęp do wysokiej jakości edukacji.



## Zaimportowane biblioteki

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib
from folium.plugins import MiniMap
from matplotlib.colors import Normalize
from shapely.geometry import Point
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from folium.plugins import MarkerCluster
from branca.colormap import linear
from folium.plugins import HeatMap

## Wczytywanie danych

In [ ]:
granicewoj = gpd.read_file('data/Granice województw/A01_Granice_wojewodztw.shp')

replace_dict = {
    'pomorskie': 'Pomorskie',
    'maloopolskie': 'Małopolskie',
    'łódzkie': 'Łódzkie',
    'warmińsko-mazurskie': 'Warmińsko-Mazurskie',
    'lubuskie': 'Lubuskie',
    'opolskie': 'Opolskie',
    'dolnośląskie': 'Dolnośląskie',
    'świętokrzyskie': 'Świętokrzyskie',
    'zachodniopomorskie': 'Zachodniopomorskie',
    'kujawsko-pomorskie': 'Kujawsko-Pomorskie',
    'śląskie': 'Śląskie',
    'podlaskie': 'Podlaskie',
    'wielkopolskie': 'Wielkopolskie',
    'mazowieckie': 'Mazowieckie',
    'lubelskie': 'Lubelskie',
    'podkarpackie': 'Podkarpackie'
}

replace_dict = {
    'pomorskie': 'Pomorskie',
    'maÅopolskie': 'Małopolskie',
    'ÅÃ³dzkie': 'Łódzkie',
    'warmiÅsko-mazurskie': 'Warmińsko-Mazurskie',
    'lubuskie': 'Lubuskie',
    'opolskie': 'Opolskie',
'dolnoÅlÄskie': 'Dolnośląskie',
    'ÅwiÄtokrzyskie': 'Świętokrzyskie',
    'zachodniopomorskie': 'Zachodniopomorskie',
    'kujawsko-pomorskie': 'Kujawsko-Pomorskie',
    'ÅlÄskie': 'Śląskie',
    'podlaskie': 'Podlaskie',
    'wielkopolskie': 'Wielkopolskie',
    'mazowieckie': 'Mazowieckie',
    'lubelskie': 'Lubelskie',
    'podkarpackie': 'Podkarpackie'
}

granicewoj['JPT_NAZWA_'] = granicewoj['JPT_NAZWA_'].replace(replace_dict)

ludnosc = gpd.read_file('polskieludy2021.gpkg')

granicewoj = granicewoj.to_crs(ludnosc.crs)

ludnosc_z_woj = gpd.sjoin(ludnosc, granicewoj, how="inner")

pop_per_wojewodztwo = ludnosc_z_woj.groupby("JPT_NAZWA_")["TOT_P_2021"].sum().reset_index()

pop_per_wojewodztwo = pop_per_wojewodztwo.sort_values(by="TOT_P_2021", ascending=False)

granicewoj["populacja"] = granicewoj["JPT_NAZWA_"].map(pop_per_wojewodztwo.set_index("JPT_NAZWA_")["TOT_P_2021"])

## Populacja województw w Polsce

In [ ]:
def format_milions(x, pos):
    return f'{int(x / 1e6)}'


cmap = plt.cm.Greens
normalize = mcolors.Normalize(vmin=pop_per_wojewodztwo["TOT_P_2021"].min(),
                              vmax=pop_per_wojewodztwo["TOT_P_2021"].max())
colors = [cmap(normalize(value)) for value in pop_per_wojewodztwo["TOT_P_2021"]]

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

axes[0].bar(pop_per_wojewodztwo["JPT_NAZWA_"], pop_per_wojewodztwo["TOT_P_2021"], color=colors, edgecolor="black")
axes[0].set_xlabel('Województwo', fontsize=12)
axes[0].set_ylabel('Populacja (mln)', fontsize=12)
axes[0].set_title('Populacja województw w Polsce', fontsize=16)
axes[0].set_xticks(range(len(pop_per_wojewodztwo["JPT_NAZWA_"])))
axes[0].set_xticklabels(pop_per_wojewodztwo["JPT_NAZWA_"], rotation=90, fontsize=10)
axes[0].yaxis.set_major_formatter(FuncFormatter(format_milions))

for idx, value in enumerate(pop_per_wojewodztwo["TOT_P_2021"]):
    axes[0].text(idx, value + 100000, f'{value / 1e6:.1f} mln', ha='center', va='bottom', fontsize=9)

granicewoj["populacja"] = granicewoj["JPT_NAZWA_"].map(pop_per_wojewodztwo.set_index("JPT_NAZWA_")["TOT_P_2021"])
granicewoj.plot(
    column="populacja",
    cmap="Greens",
    legend=True,
    edgecolor="black",
    ax=axes[1]
)

axes[1].set_title('Mapa populacji województw w Polsce', fontsize=16)
axes[1].set_axis_off()

plt.tight_layout()
plt.show()

## Ilość placówek edukacyjnych z podziałem na wojewodztwa.

In [ ]:
szkoly = gpd.read_file("polska_szkoly.gpkg")

szkoly_copy = szkoly.to_crs(granicewoj.crs).copy()
szkoly_woj = gpd.sjoin(szkoly_copy, granicewoj, how="inner")
liczba_szkol = szkoly_woj.groupby("JPT_NAZWA_").size().reset_index(name="liczba_szkol")

granicewoj["liczba_szkol"] = granicewoj["JPT_NAZWA_"].map(liczba_szkol.set_index("JPT_NAZWA_")["liczba_szkol"])
granicewoj = granicewoj.sort_values(by="liczba_szkol", ascending=False)
cmap = plt.cm.Blues
normalize = mcolors.Normalize(vmin=granicewoj["liczba_szkol"].min(),
                              vmax=granicewoj["liczba_szkol"].max())
colors = [cmap(normalize(value)) for value in granicewoj["liczba_szkol"]]

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

axes[0].bar(granicewoj["JPT_NAZWA_"], granicewoj["liczba_szkol"], color=colors, edgecolor="black")
axes[0].set_title("Liczba szkół w województwach", fontsize=16)
axes[0].set_ylabel("Liczba szkół", fontsize=12)
axes[0].set_xticks(range(len(granicewoj["JPT_NAZWA_"])))
axes[0].set_xticklabels(granicewoj["JPT_NAZWA_"], rotation=45, ha="right", fontsize=10)

bars = axes[0].containers[0]
for bar, value in zip(bars, granicewoj["liczba_szkol"]):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width() / 2.0, height, f"{value}",
                 ha='center', va='bottom', fontsize=9)

granicewoj.plot(
    column="liczba_szkol",
    cmap="Blues",
    legend=True,
    edgecolor="black",
    ax=axes[1]
)
axes[1].set_title("Mapa liczby szkół w województwach", fontsize=16)
axes[1].set_axis_off()

plt.tight_layout()
plt.show()

## Liczba szkół przypadajacych na jednego mieszkańca w danym województwie.

In [ ]:
granicewoj["szkol_na_mieszkanca"] = granicewoj["populacja"] / granicewoj["liczba_szkol"]
granicewoj = granicewoj.sort_values(by="szkol_na_mieszkanca", ascending=False)

cmap = plt.cm.Reds
normalize = mcolors.Normalize(vmin=granicewoj["szkol_na_mieszkanca"].min(),
                              vmax=granicewoj["szkol_na_mieszkanca"].max())
colors = [cmap(normalize(value)) for value in granicewoj["szkol_na_mieszkanca"]]

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

axes[0].bar(granicewoj["JPT_NAZWA_"], granicewoj["szkol_na_mieszkanca"], color=colors, edgecolor="black")
axes[0].set_title("Liczba mieszkańców na szkołę", fontsize=16)
axes[0].set_ylabel("Liczba mieszkańców na szkołę", fontsize=12)
axes[0].set_xticks(range(len(granicewoj["JPT_NAZWA_"])))
axes[0].set_xticklabels(granicewoj["JPT_NAZWA_"], rotation=45, ha="right", fontsize=10)
axes[0].set_ylim([900, None])

bars = axes[0].containers[0]
for bar, value in zip(bars, granicewoj["szkol_na_mieszkanca"]):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width() / 2.0, height, f"{value:.0f}",
                 ha='center', va='bottom', fontsize=9)

granicewoj.plot(
    column="szkol_na_mieszkanca",
    cmap="Reds",
    legend=True,
    edgecolor="black",
    ax=axes[1]
)

axes[1].set_title("Liczba mieszkańców na szkołę", fontsize=16)
axes[1].set_axis_off()

plt.tight_layout()
plt.show()

## Procent udziału szkół publicznych i prywatnych zależnie od poziomu edukacji.

In [ ]:
statystyki = szkoly.groupby(["levels", "public_private"]).size().reset_index(name="liczba_szkol")

ogolna_liczba_szkol = statystyki.groupby("levels")["liczba_szkol"].sum().reset_index(name="ogolna_liczba_szkol")

prywatne_szkoly = statystyki[statystyki["public_private"] == "PRIVATE"]

ogolna_liczba_przedszkoli = ogolna_liczba_szkol[ogolna_liczba_szkol["levels"].isin(["0"])]
prywatne_przedszkola = prywatne_szkoly[prywatne_szkoly["levels"].isin(["0"])]

ogolna_liczba_podst = ogolna_liczba_szkol[ogolna_liczba_szkol["levels"].isin(["1"])]
prywatne_podst = prywatne_szkoly[prywatne_szkoly["levels"].isin(["1"])]

ogolna_liczba_srednich = ogolna_liczba_szkol[ogolna_liczba_szkol["levels"].isin(["2"])]
prywatne_srednie = prywatne_szkoly[prywatne_szkoly["levels"].isin(["2"])]

proc_przedsz = prywatne_przedszkola['liczba_szkol'].iloc[0] / ogolna_liczba_przedszkoli['ogolna_liczba_szkol'].iloc[
    0] * 100

proc_podst = prywatne_podst['liczba_szkol'].iloc[0] / ogolna_liczba_podst['ogolna_liczba_szkol'].iloc[0] * 100

proc_sred = prywatne_srednie['liczba_szkol'].iloc[0] / ogolna_liczba_srednich['ogolna_liczba_szkol'].iloc[0] * 100

data = {
    'Level': ['Przedszkola', 'Szkoły Podstawowe', 'Szkoły Średnie'],
    'Percentage': [proc_przedsz, proc_podst, proc_sred]
}

df = pd.DataFrame(data)

plt.figure(figsize=(10, 6))
plt.bar(data['Level'], data['Percentage'], color=['green', 'blue', 'red'], edgecolor="black")
plt.xlabel('Poziom edukacji')
plt.ylabel('Procent szkół prywatnych')
plt.title('Procentowy udział szkół prywatnych na różnych poziomach edukacji')
# plt.ylim(0, 100)  # Set y-axis limit to 100%
plt.tight_layout()
plt.show()


## Procentowy udział placówek prywatnych i publicznych w danym województwie.

In [ ]:
liczba_pryw = szkoly['public_private'].value_counts()
# liczba_pryw
# Grupa danych według województw i typu szkół (publiczne/prywatne)
liczba_szkol_typ = szkoly_woj.groupby(["JPT_NAZWA_", "public_private"]).size().reset_index(name="liczba_szkol")

# Lista unikalnych województw
wojewodztwa = liczba_szkol_typ["JPT_NAZWA_"].unique()

# Kolory dla typów szkół
colors = {'PUBLIC': '#66c2a5', 'PRIVATE': '#fc8d62'}

# Tworzenie wykresów kołowych dla każdego województwa
plt.figure(figsize=(15, 20))  # Dopasowanie rozmiaru figury
for i, woj in enumerate(wojewodztwa, 1):
    # Filtrowanie danych dla danego województwa
    dane_woj = liczba_szkol_typ[liczba_szkol_typ["JPT_NAZWA_"] == woj]

    # Dane do wykresu kołowego
    labels = dane_woj["public_private"]
    sizes = dane_woj["liczba_szkol"]

    plt.subplot(4, 4, i) 
    plt.pie(
        sizes,
        labels=None,  # Ukrywamy etykiety na samym wykresie
        autopct='%1.1f%%',
        startangle=90,
        colors=[colors[label] for label in labels]
    )
    plt.title(woj, fontsize=12)  # Zwiększenie czcionki tytułu

# Dodanie wspólnej legendy dla wszystkich wykresów
plt.legend(
    labels=colors.keys(),
    loc='upper center',
    bbox_to_anchor=(0.5, -0.1),  # Pozycja poniżej wykresów
    ncol=2,
    title='Typ szkoły',
    fontsize=10,
    title_fontsize=12
)

# Dostosowanie layoutu
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Zmniejszenie marginesów, aby zmieścić całość
plt.suptitle('Procentowy udział szkół publicznych i prywatnych w województwach', fontsize=16)
plt.subplots_adjust(bottom=0.2)  # Więcej miejsca na dole dla legendy
plt.show()

In [ ]:
m3 = folium.Map(location=[52.04, 19.28], zoom_start=7,
                tiles='OpenStreetMap',
                control_scale=True)

granicewoj_geojson = granicewoj.to_crs(epsg=4326).copy()

for column in granicewoj_geojson.select_dtypes(include=['datetime']):
    granicewoj_geojson[column] = granicewoj_geojson[column].astype(str)

granicewoj_geojson
granicewoj_geojson = granicewoj_geojson.rename(columns={'JPT_NAZWA_': 'Nazwa', 'populacja': 'Populacja', 'liczba_szkol': 'Liczba szkół', 'szkol_na_mieszkanca': 'Liczba szkół na mieszkańca'})

popup = folium.GeoJsonPopup(fields=["Nazwa", "Populacja", "Liczba szkół", "Liczba szkół na mieszkańca"],)


folium.GeoJson(granicewoj_geojson,
               highlight_function=lambda feature: {
                   "fillColor":( "yellow" if "e" in feature["properties"]["Nazwa"].lower() else "#ffff00"),
               },
               style_function=lambda x: {
                   "color": "black",
                   "weight": 2,
                   "dashArray": "5, 5",
               },

               popup=popup,
               ).add_to(m3)

folium.LayerControl().add_to(m3)

m3

## Mapa rozmieszczenia szkół w kraju i zagęszczenia uczniów w szkołach

In [ ]:
m = folium.Map(location=[52.04, 19.28], zoom_start=7,
               tiles='OpenStreetMap',
               control_scale=True)

szkoly['email'] = szkoly['email'].fillna('Brak')
szkoly['tel'] = szkoly['tel'].fillna('Brak')
szkoly['url'] = szkoly['url'].fillna('Brak')

def icon_color(observer):
    colors = {
        '0': 'green',
        '1': 'blue',
        '2': 'red'
    }
    return colors.get(observer)


def icon_sign(amount):
    return 'angle-down' if int(amount) <= 100 else 'angles-down'


def education_level(amount):
    poziom = {
        '0': 'Przedszkolny',
        '1': 'Podstawowy',
        '2': 'Średni'
    }
    return poziom.get(amount)


cluster_layer = folium.FeatureGroup(name="Rozmieszczenie szkół w kraju", control=True, overlay=True, show=False)
marker_cluster = MarkerCluster().add_to(cluster_layer)

for _, row in szkoly.iterrows():
    coord = (row['geometry'].y, row['geometry'].x)

    nazwa_szkoly = row['name']
    poziom_szkoly = row['levels']
    liczba_uczniow = row['enrollment']
    pryw_pub = row['public_private']


    folium.Marker(
        location=coord,
        popup=f"""
    <div style="width: 240px; height: 200px; overflow: auto;">
        <b>Nazwa: </b> {nazwa_szkoly}<br>
        <b>Poziom edukacji: </b> {education_level(poziom_szkoly)}<br>
        <b>Liczba uczniów: </b> {liczba_uczniow}<br>
        <b>Typ: </b> {pryw_pub}<br>
        <b>Adres: </b> {row['street']} {row['house_number']}, {row['postcode']}<br>
        <b>Miejscowość: </b> {row['city']}<br>
        <b>Email: </b> {row['email']}<br>
        <b>Telefon: </b> {row['tel']}<br>
        <b>Strona internetowa: </b> {row['url']}<br>

    </div>
""",
        icon=folium.Icon(color=icon_color(poziom_szkoly), icon=icon_sign(liczba_uczniow), prefix='fa'),
    ).add_to(marker_cluster)

cluster_layer.add_to(m)

szkoly_bez_chmur = szkoly[~szkoly['email'].str.contains('chmurze.org')].copy()

szkoly_bez_chmur['enrollment'] = pd.to_numeric(szkoly_bez_chmur['enrollment'], errors='coerce')

heat_data = szkoly_bez_chmur[['lat', 'lon', 'enrollment']].values.tolist()

heatmap_layer = folium.FeatureGroup(name="Zagęszczenie uczniów", control=True, overlay=True, show=False)

HeatMap(heat_data, blur=15, max_zoom=10, min_opacity=0.15).add_to(heatmap_layer)

heatmap_layer.add_to(m)

folium.LayerControl().add_to(m)

m

## Mapa szkół podstawowych bez dostępu do szkół średnich w promieniu 15 km

In [ ]:
m2 = folium.Map(location=[52.04, 19.28], zoom_start=7,
                tiles='OpenStreetMap',
                control_scale=True)

szkoly_bez_srednich = []
counter = 0
szkoly_podstawowe = szkoly[szkoly['levels'] == '1'].to_crs(epsg=2180).copy()
szkoly_srednie = szkoly[szkoly['levels'] == '2'].to_crs(epsg=2180)
szkoly_srednie = szkoly_srednie[szkoly_srednie['public_private'] == 'PUBLIC'].to_crs(epsg=2180)

szkoly_podstawowe['buffer_15km'] = szkoly_podstawowe.geometry.buffer(15000)

for i, row in szkoly_podstawowe.iterrows():
    if szkoly_srednie[szkoly_srednie.geometry.within(row['buffer_15km'])].empty:
        szkoly_bez_srednich.append(row)
        counter = counter + 1

szkoly_bez_srednich_df = pd.DataFrame(szkoly_bez_srednich)
szkoly_bez_srednich_df = szkoly_bez_srednich_df.drop(columns='buffer_15km').copy()
szkoly_bez_srednich_gdf = gpd.GeoDataFrame(szkoly_bez_srednich_df, crs='EPSG:4326')
szkoly_bez_srednich_gdf["geometry"] = gpd.points_from_xy(szkoly_bez_srednich_gdf['lon'], szkoly_bez_srednich_gdf['lat'])

tooltip = folium.features.GeoJsonTooltip(
    fields=("name",),
    aliases=("Nazwa szkoły: ",)
)

points_geojson = folium.features.GeoJson(szkoly_bez_srednich_gdf, tooltip=tooltip,
                                         name="Rozmieszczenie szkol w polsce",
                                         control=True)
odlegle = folium.FeatureGroup(name="Romieszczenie szkół w kraju", control=False, overlay=True, show=True)
points_geojson.add_to(odlegle)
odlegle.add_to(m2)

MiniMap(zoom_level_offset=-4,width=200, height=200).add_to(m2)

folium.LayerControl().add_to(m2)

m2

## Mapa odległości od liceów wysoko plasujących się w rankingu

In [ ]:
selected_school_ids = ["PL_25801", "PL_4735", "PL_15721", "PL_27858", "PL_29583", "PL_89690", "PL_25956", "PL_11474",
                       "PL_27005", "PL_40982", "PL_121553", "PL_7713", "PL_39649", "PL_31606", "PL_56095", "PL_27748",
                       "PL_34199", "PL_13376", "PL_30251", "PL_25682"]
selected_schools = szkoly[szkoly["id"].isin(selected_school_ids)]
szkoly.loc[szkoly["id"].isin(selected_school_ids), "miejsce"] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                                                                 17, 18, 19, 20]

filtered_szkoly = szkoly.dropna(subset=['miejsce'])

punkty_ref = []

for _, szkola in filtered_szkoly.iterrows():
    punkty_ref.append(szkola['geometry'])

punkty_ref_gdf = gpd.GeoDataFrame(geometry=punkty_ref, crs="EPSG:4326").to_crs(epsg=3857)

# punkty_ref_gdf

punkty = gpd.read_file("polskieludy2021.gpkg")

punkty = punkty.to_crs(epsg=3857)
punkty_ref_gdf = punkty_ref_gdf.to_crs(epsg=3857)


def calculate_min_distance(geometry, reference_points):
    return reference_points.geometry.distance(geometry).min()


punkty["distance_km"] = punkty.geometry.apply(
    lambda geom: calculate_min_distance(geom, punkty_ref_gdf)
) / 1000

colormap = matplotlib.colormaps["plasma"]

norm = Normalize(vmin=0, vmax=punkty["distance_km"].max() * 0.8)

punkty["color"] = punkty["distance_km"].apply(lambda d: colormap(norm(d)))

plt.figure(figsize=(15, 15))

punkty = punkty.to_crs(epsg=4326)
punkty_ref_gdf = punkty_ref_gdf.to_crs(epsg=4326)
granicewoj_copy = granicewoj.to_crs(epsg=4326).copy()

base = punkty.plot(
    color=punkty["color"], alpha=0.6, markersize=10, legend=True, label="Punkty"
)

granicewoj_copy.plot(
    ax=base,
    edgecolor="black",
    linewidth=0.5,
    facecolor="none",
    label="Granice województw"
)

punkty_ref_gdf.plot(ax=base, color="purple", marker="x", markersize=50, label="Punkty referencyjne")
plt.axis("off")
plt.title("Odległość od liceów plasujących się wysoko w rankingu")

# plt.legend()
plt.tight_layout()
plt.show()


# Wnioski

1. Rozmieszczenie szkół w Polsce jest nierównomierne i zależne od województwa. Województwa o dużej populacji, takie jak Mazowieckie czy Śląskie, posiadają najwięcej szkół. Z kolei województwa o mniejszej populacji, takie jak Opolskie czy Lubuskie, mają ich znacznie mniej.

    - Liczba szkół w województwach nie zawsze odpowiada proporcjonalnie liczbie mieszkańców – niektóre regiony cierpią na większe obciążenie szkół w stosunku do populacji, co może wpływać na jakość edukacji.

   - Województwa o niższej gęstości zaludnienia mają mniej placówek edukacyjnych, co może ograniczać dostępność edukacji dla mieszkańców obszarów wiejskich.
  
2. Obliczenie liczby mieszkańców przypadających na jedną szkołę ukazuje dużą różnicę między województwami:
    - Województwa takie jak Podlaskie i Warmińsko-Mazurskie mają niski wskaźnik mieszkańców na szkołę, co oznacza lepszy dostęp do placówek w przeliczeniu na populację.
      
    - W województwach takich jak Śląskie i Mazowieckie duża liczba ludności powoduje wysoki wskaźnik mieszkańców przypadających na jedną szkołę.
  
    - Duże wskaźniki mieszkańców przypadających na szkołę mogą świadczyć o przeciążeniu placówek i konieczności rozbudowy sieci szkół w tych
      obszarach.
  
    - Województwa z niższą gęstością zaludnienia dysponują mniejszą liczbą placówek, ale lepszym wskaźnikiem mieszkańców na szkołę.
    
3. Szkoły publiczne zdecydowanie dominują w Polsce, stanowiąc około 75% wszystkich placówek. Szkoły prywatne występują w znacznie mniejszej liczbie, szczególnie w mniejszych województwach, co pokazuje brak równowagi między sektorami edukacji.

    - Szkoły prywatne częściej pojawiają się na poziomie przedszkolnym niż w szkołach podstawowych czy średnich.

5. Analiza odległości szkół podstawowych od średnich pokazuje, że istnieją obszary o trudnym dostępie do szkół średnich (brak szkoły średniej w promieniu 15 km), szczególnie w województwach o dużych odległościach geograficznych między miejscowościami, takich jak Podkarpackie, Warmińsko-Mazurskie czy Lubelskie.
    - Problem dostępu do szkół średnich bardziej dotyczy terenów wiejskich o niskim zagęszczeniu ludności. W takich miejscach uczniowie są zmuszeni do codziennych dalekich dojazdów.
    - Skupienie szkół średnich w miastach skutkuje ograniczonym dostępem do edukacji na poziomie średnim dla mieszkających z dala od większych miast.
6. Najlepsze licea (top 20 rankingu perspektyw) są skoncentrowane głównie w dużych miastach (Warszawa, Kraków, Poznań, Wrocław). Uczniowie z mniejszych miejscowości muszą pokonywać znaczne dystanse, aby uzyskać dostęp do tej klasy edukacji.
    - Ta nierównowaga może zwiększać dysproporcje edukacyjne, ponieważ uczniowie z mniejszych miast i terenów wiejskich mają mniejsze szanse na naukę w najlepszych szkołach.

## Podsumowanie

Przeprowadzona przez nas analiza danych oraz sporządzenie odpowiednich map pozwoliła na potwierdzenie naszych hipotez, dotyczących 
nierównomiernego rozmieszczenia szkół w Polsce. Udało nam się pokazać, że znaczna ilość szkół jest skupiona wśród dużych miast, co ogranicza
dostęp szkół średnich dla mniejszych miast i wsi. Taka sytuacja może prowadzić do pogłębiania nierówności edukacyjnych, zwłaszcza w kontekście dostępu do najlepszych szkół. Uczniowie z terenów wiejskich i małych miejscowości często muszą pokonywać duże odległości, co może wpływać na ich wyniki i decyzje o przyszłości związanej z edukacją. W przyszłości warto rozważyć działania wspierające równomierny rozwój infrastruktury edukacyjne